In [2]:
!pip uninstall keyring --yes

Found existing installation: keyring 23.13.1
Uninstalling keyring-23.13.1:
  Successfully uninstalled keyring-23.13.1


In [3]:
!pip install keyring

  Using cached keyring-23.13.1-py3-none-any.whl (37 kB)
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install --upgrade snowflake-connector-python

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [5]:
import pandas as pd
import snowflake.connector

In [6]:
ctx = snowflake.connector.connect(
    user='yashrajsingh',
    password='Unlock2?',
    account='fk72744.us-east-2.aws',
    role = 'ACCOUNTADMIN',
    warehouse = 'COMPUTE_WH',
    database = 'HRPCI', 
    schema = 'GRATICULESOW3'
)

In [20]:
EVU_STRINGS = ['%Evusheld%', '%tixagev%', '%cilgav%']

In [21]:
EVU_NDC = ['0310744202','0310889501','0310106101','0310886102','00310744202','00310889501','00310106101','00310886102']

In [22]:
EVU_NDC = ['0310744202','0310889501','0310106101','0310886102','00310744202','00310889501','00310106101','00310886102']

In [23]:
evu_exposure_subqueries = [
    """
    SELECT pr.DataOwnerId, 
    p.MasterPatientId,
    COALESCE(pr.PxDate, e.EncounterDate, e.AdmitDate, e.DischargeDate) AS exposuredate,
    'PROCEDURES' AS exposuretype
    FROM VW_DEID_PROCEDURES pr
    INNER JOIN VW_DEID_PATIENTS p ON pr.DataOwnerId = p.DataOwnerId AND pr.PatientId = p.PatientId
    LEFT JOIN VW_DEID_ENCOUNTERS e ON pr.DataOwnerId = e.DataOwnerId AND pr.EncounterId = e.EncounterId
    WHERE 
        COALESCE(pr.PxDate, e.EncounterDate, e.AdmitDate, e.DischargeDate) BETWEEN TO_DATE(%(START_DATE)s) AND TO_DATE(%(END_DATE)s)
        AND (pr.SourcePxCodeDescription ILIKE ANY (%(NAME)s)
        OR pr.PxCode ILIKE ANY (%(PX)s))
    """,
    """
    SELECT b.DataOwnerId, 
    p.MasterPatientId,
    COALESCE(b.ServiceDate, e.EncounterDate, e.AdmitDate, e.DischargeDate) AS exposuredate,
    'BILLINGS' AS exposuretype
    FROM VW_DEID_PATIENTS p
    INNER JOIN VW_DEID_BILLINGS b ON p.DataOwnerId = b.DataOwnerId AND b.PatientId = p.PatientId 
    LEFT JOIN VW_DEID_ENCOUNTERS e ON b.DataOwnerId = e.DataOwnerId AND b.EncounterId = e.EncounterId
    WHERE
        COALESCE(b.ServiceDate, e.EncounterDate, e.AdmitDate, e.DischargeDate) BETWEEN TO_DATE(%(START_DATE)s) AND TO_DATE(%(END_DATE)s)
        AND (b.BillingProcedureDescription ILIKE ANY (%(NAME)s) 
        OR b.MedicationName ILIKE ANY (%(NAME)s)
        OR b.BillingProcedureCode ILIKE ANY (%(PX)s))
    """,
    """
    SELECT mo.DataOwnerId AS DataOwnerId,
    p.MasterPatientId,
    COALESCE(mo.OrderDate, e.AdmitDate, e.EncounterDate, e.DischargeDate) AS exposuredate,
    'MEDORDERS' AS exposuretype
    FROM VW_DEID_MEDORDERS mo
    INNER JOIN VW_DEID_PATIENTS p ON mo.DataOwnerId = p.DataOwnerId AND mo.PatientId = p.PatientId
    LEFT JOIN VW_DEID_ENCOUNTERS e ON mo.DataOwnerId = e.DataOwnerId AND mo.EncounterId = e.EncounterId
    WHERE
        COALESCE(mo.OrderDate, e.AdmitDate, e.EncounterDate, e.DischargeDate) BETWEEN TO_DATE(%(START_DATE)s) AND TO_DATE(%(END_DATE)s)
        AND (mo.SourceRxMedName ILIKE ANY (%(NAME)s)
        OR (mo.SourceRxCodeType = 'NDC' AND REPLACE(mo.SourceRxCode, '-') ILIKE ANY (%(NDC)s)))

    """,
    """
    SELECT mf.DataOwnerId AS DataOwnerId,
    p.MasterPatientId,
    mf.FilledDate AS exposuredate,
    'MEDFILLS' AS exposuretype
    FROM VW_DEID_MEDFILLS mf
    INNER JOIN VW_DEID_PATIENTS p ON mf.DataOwnerId = p.DataOwnerId AND mf.PatientId = p.PatientId
    WHERE 
        mf.FilledDate BETWEEN TO_DATE(%(START_DATE)s) AND TO_DATE(%(END_DATE)s)
        AND (mf.SourceDrugName ILIKE ANY (%(NAME)s)
        OR REPLACE(mf.NDC, '-') ILIKE ANY (%(NDC)s))
    """,
    """
    SELECT ma.DataOwnerId AS DataOwnerId,
    p.MasterPatientId AS MasterPatientId,
    COALESCE(ma.MedAdminStartDate, e.AdmitDate, e.EncounterDate, e.DischargeDate) AS exposuredate,
    'MEDADMINS' AS exposuretype
    FROM VW_DEID_MEDADMINS ma
    INNER JOIN VW_DEID_PATIENTS p ON ma.DataOwnerId = p.DataOwnerId AND ma.PatientId = p.PatientId
    LEFT JOIN VW_DEID_ENCOUNTERS e ON ma.DataOwnerId = e.DataOwnerId AND ma.EncounterId = e.EncounterId
    WHERE 
        COALESCE(ma.MedAdminStartDate, e.AdmitDate, e.EncounterDate, e.DischargeDate) BETWEEN TO_DATE(%(START_DATE)s) AND TO_DATE(%(END_DATE)s)
        AND (ma.SourceMedAdminMedName ILIKE ANY (%(NAME)s)
        OR REPLACE(ma.NDC, '-') ILIKE ANY (%(NDC)s))
    """,
]

In [24]:
evu_exposure_query = "UNION".join(evu_exposure_subqueries)

In [28]:
exposure_df = ctx.cursor().execute(evu_exposure_query, {
    'NAME': EVU_STRINGS, 
    'NDC': EVU_NDC,
}).fetch_pandas_all()

KeyError: 'START_DATE'

In [7]:
df1 = ctx.cursor().execute("SELECT * FROM VW_DEID_MEDADMINS limit 10").fetch_pandas_all()

In [8]:
df1

,MEDADMINID,PATIENTID,ENCOUNTERID,MEDORDERID,DATAOWNERID,PROVIDERID,FACILITYID,MEDADMINSTARTDATE,MEDADMINSTARTTIME,MEDADMINSTOPDATE,...,MEDADMINSOURCE,PHARMACYID,SOURCEMEDADMINMEDNAME,RECORDCREATEDON,RECORDUPDATEDON,NDC,RECORDDELETEDON,ISACTIVE,ISDELETED,DATAREFRESHID
0,5f45ceda27824516dde2bf3d3eebf8f32e9ffdf6d04478...,ab045b330afcaf384a08a04525f27512b8b69c26ff1ae4...,d37614ae8b0c6e0b54611807b3ddb71bdfa714dfa473f4...,9d2de4a42fadbfa8e5d0640585b4d65deba05156234d78...,c877ef723bbc0de29171dcaa993fe3b86064bb0d0d7be4...,222ED1E7-8516-4B5D-AF17-3112C1C243C9,None,2021-04-01,00:00:00,None,...,UN,None,Acyclovir Tab 800 MG,2022-07-20 11:18:50.830,2023-02-13 00:30:48.423,31722-778-01,NaT,1,0,2
1,6cefdfeb4b9a900f133ffbb9437d92b4ace666f16720c5...,c43e376985409828912a9be35efa190840245eb7daf5b1...,None,e1e948f359906b16a10c8b3b41444a99faf10ff4b16a7a...,5540c5b22e7435d6eb2a63e3a80b623d75eb34812013cb...,AD750169-0D37-4765-A5DF-99BB0B6A5EF1,None,2020-02-02,00:00:00,None,...,UN,None,Speciality Vitamin Product Tab,2021-06-22 23:17:33.848,2023-02-10 23:12:57.274,17204-434-40,NaT,1,0,2
2,25e4c41207cbf2b0bb087f344fffb70070ec42266c08ef...,aba91ead37f3334e0db28844049599cb372949a94ec274...,d4c90e79542b1d68a4a24dc98ce0e938b0a4ac53bd0274...,8c2d851643c24c15da5a8b803dc309f973d537b99eda35...,c877ef723bbc0de29171dcaa993fe3b86064bb0d0d7be4...,C54ACFFF-4A5A-4A70-9A12-9DB6C0160F9D,None,2023-02-18,00:00:00,None,...,UN,None,Losartan Potassium Tab 25 MG,2023-02-26 04:36:24.544,2023-02-26 04:50:01.506,63739-673-10,NaT,1,0,2
3,df3a4f81ed212782254b60a0a9570d789e62b185fa75bd...,c327af8c4f6709017eec1f8f70594d6662230be3bc4cd3...,a8c1418464b473037744e9a297948f80446df5cb3741ad...,e3c0db8525753f99268eb14bd0016a3cdeceb6f06bfae3...,c877ef723bbc0de29171dcaa993fe3b86064bb0d0d7be4...,FCEB13C9-39DA-43B9-9410-86AF9B26B3F0,None,2022-03-26,00:00:00,None,...,UN,None,Oxycodone HCl Tab 5 MG,2022-10-05 13:56:46.186,2023-02-13 00:30:48.423,42858-001-10,NaT,1,0,2
4,363c64bef4c197dad7dc32b4dafa5d0f659f41fe5318d1...,c327af8c4f6709017eec1f8f70594d6662230be3bc4cd3...,a8c1418464b473037744e9a297948f80446df5cb3741ad...,183c100f490802c47a747c06c12deb0974ad93ab5ff5b5...,c877ef723bbc0de29171dcaa993fe3b86064bb0d0d7be4...,FCEB13C9-39DA-43B9-9410-86AF9B26B3F0,None,2022-03-26,00:00:00,None,...,UN,None,Sodium Chloride Flush IV Soln 0.9%,2022-10-05 13:56:46.186,2023-02-13 00:30:48.423,8290-306546,NaT,1,0,2
5,3d949ed0ef3f776d25db5832d2b6dea01904ca413544db...,476c7bebe333d74c2776588d5c97f2ed8fe44d51d86eea...,None,c723c837b9373bbe9c643dbf10ea961f8b5f7d7734d147...,137301f961d1fd77b33c529c9be609e52c35aca429583f...,AE5300DF-5D76-4E0A-B9CF-D35DC305DF01,None,2022-03-03,00:00:00,None,...,UN,None,Nitroglycerin TL Soln 0.4 MG/SPRAY (400 MCG/SP...,2022-03-06 13:03:14.760,2023-02-10 23:12:57.274,45802-210-01,NaT,1,0,2
6,ac61a6857f11e19925118d1b169ff68bc355ee2ef0c188...,8cdf9a9ad59bf73c5ed25f56d03c5e944f835ce2f2c7e3...,73ab0402f6349e2d1d5ae16ed9456979340f3eafe6854b...,2e79fe683581d7878c5c74aed5cba2f1e201fecdc2c056...,c877ef723bbc0de29171dcaa993fe3b86064bb0d0d7be4...,FF2B1888-5403-4362-BB48-4E9571046EC8,None,2021-11-29,00:00:00,None,...,UN,None,None,2022-08-07 07:58:53.618,2023-02-13 00:30:48.423,63020-049-01,NaT,1,0,2
7,c9dd532350a72446d23711ce11a36503a48cdf101cbd6b...,e8ec47058e0de0a83e162ce8b5b18fe119d48663454fdf...,5f35a189d7e06e028f7e59992a60cf2d81899802ce2a05...,a9fa00453862beb55249c7846790210d2ae7ffec4d43dd...,137301f961d1fd77b33c529c9be609e52c35aca429583f...,AE6804C3-0003-4431-B819-1BDB9DF2B02B,None,2020-07-14,00:00:00,None,...,UN,None,Vancomycin HCl For Oral Soln 50 MG/ML (Base Eq...,2021-06-22 23:17:33.848,2023-02-10 23:12:57.274,99908-0321-1,NaT,1,0,2
8,0df5beb45d4b4e40fe5eed74d7fd5d8c38e5ae01acc851...,8800a3ac7fc849574e00643509cebca32f5e948a4bbf1c...,4586cf143660b0c869c9c05b1a095a75adfa95df24634b...,c27cd46994540d07f1e164e798480569682b1a2a16f46e...,c877ef723bbc0de29171dcaa993fe3b86064bb0d0d7be4...,06C9C038-7A5B-4975-9C41-77EF85EE6C51,None,2022-04-16,00:00:00,None,...,UN,None,Fentanyl Citrate Preservative F

NameError: name 'printf' is not defined

In [ ]:
ex